In [1]:
from packages import *
import numpy as np
import tensorflow as tf
from tensorflow.data.experimental import AUTOTUNE

In [2]:
epsilon = 1e-7
fp = 'float32'
tf.config.gpu.set_per_process_memory_growth(True)
tf.config.gpu.set_per_process_memory_fraction(.85)
tf.keras.backend.set_floatx(fp)
tf.keras.backend.set_epsilon(epsilon)
tf.keras.backend.set_image_data_format('channels_last')

In [3]:
to_ix = True
if to_ix:
    all_key_to_ix = load_pickle(os.path.join(GLOVE_DIR, 'glove-byte-float32_to_ix.pkl'))
    j = [2]
    keys_to_ix = {}
    keys_to_ix[NULL_KEY.encode()] = 0
    keys_to_ix[UNK_KEY.encode()] = 1 
#ASSUMING ix 0 is for NULL and 1 for UNK

In [4]:
# Helperfunctions to make your feature definition more readable
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[*value]))

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[*value]))

In [5]:
def serialize_example(review, to_ix):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    
    
    
    feature = {
      'review': _int64_feature(review) if to_ix else _bytes_feature(review),
    }

    # Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [6]:
chunk_count = 100

In [7]:
def _serialize(review, to_ix=False):
    if to_ix:
        ix = []
        for word in review:
            if word in keys_to_ix:
                ix.append(keys_to_ix[word])
            else:
                keys_to_ix[word] = j[0]
                ix.append(j[0])
                j[0]+=1
        review = ix   
    return serialize_example(review, to_ix)

In [10]:
review_paths = [os.path.join(DATASET_DIR, 'preprocessed', 'npy', 'review-text-{:02d}.npy'.format(i)) for i in range(chunk_count)] #100 files

In [ ]:
for i in range(chunk_count):
    chunk = np.load(review_paths[i],allow_pickle=False)
    chunk_tf = [_serialize(x, to_ix = to_ix) for x in chunk]
    chunk_ds = tf.data.Dataset.from_tensor_slices(chunk_tf)
    writer_path = os.path.join(DATASET_DIR, 'preprocessed', 'tfrecord', 'ix' if to_ix else 'xext',
                               'review-{}-{:02d}.tf'.format('ix' if to_ix else 'text', i))
    writer = tf.data.experimental.TFRecordWriter(writer_path)
    writer.write(chunk_ds)
    print(i+1, end=', ')

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 

In [12]:
# for x in chunk_ds:break
# writer = tf.data.experimental.TFRecordWriter(writer_path)
# writer.write(chunk_ds)

In [14]:
def _parse_function(proto, to_ix):
    # define your tfrecord again. Remember that you saved your image as a string.
    keys_to_features = {'review': tf.io.FixedLenFeature([300,], tf.int64) if to_ix else tf.io.FixedLenFeature([300,], tf.string),}
    
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    return parsed_features 

In [15]:
dataset = tf.data.TFRecordDataset(writer_path)

In [16]:
dataset = dataset.map(lambda x: _parse_function(x, to_ix)['review'], num_parallel_calls=8)

In [17]:
for x in dataset:
    break